In [1]:
import collections
import itertools
import json
import datetime
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm

In [2]:
LANGUAGES = [
     #"rust",
    "python",
    "javascript",
    "java",
    "ruby",
    "r",
]

# Probability of a random post being a novelty

In [ ]:
novelty_plots = {}
for language in LANGUAGES:
    print(f"Processing {language}...")
    path_to_lib = f"data/results/{language}"
    
    with open(f"{path_to_lib}/{language}_{language}_post_stats.json") as handle:
        post_stats = json.load(handle)
    sorted_post_stats = sorted(post_stats, key=lambda item:  datetime.datetime.strptime(item['date'], "%Y-%m-%dT%H:%M:%S.%f"))
    
    with open(f"{path_to_lib}/{language}_pairs_first_dates.json") as handle:
        pairs_first_dates = json.load(handle)
    
    with open(f"{path_to_lib}/{language}_all_pairs_dates.json") as handle:
        pairs_dates = json.load(handle)
    
    annual_post_count = collections.defaultdict(int)
    annual_novelty_count = collections.defaultdict(int)
    
    for post_stat in tqdm.tqdm(sorted_post_stats):
        year = post_stat["date"][:4]
        annual_post_count[year] += 1
        novelty_count = 0
        for imp1, imp2 in itertools.combinations(post_stat["imports"], 2):
            canonical_pair_name = "|".join(sorted([imp1, imp2]))
            if pairs_first_dates[canonical_pair_name]["id"] == post_stat["id"]:
                novelty_count += 1
        if novelty_count > 0:
            annual_novelty_count[year] += 1

    xs = list(annual_post_count.keys())
    ys = [n/max(1, t) for n, t in zip(annual_novelty_count.values(), annual_post_count.values())]
    novelty_plots[language] = ((xs, ys))

Processing python...


In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(10)
f.set_figwidth(20)
for lang in novelty_plots:
    ax.plot(*novelty_plots[lang], "-*", label=lang)
ax.legend(fontsize=12)
ax.set_ylabel("Probability of a Post being a Novelty", fontsize=14)
ax.grid(alpha = 0.5)
plt.xticks(rotation=70)
ax.set_ylim(ymin=0)
ax.set_xlim(["2010", "2022"])
plt.xticks(size=15)
plt.yticks(size=15)


# Probability of a random post being a valuable novelty

In [ ]:
VALUE_THRESHOLD_RATIO = 0.0001
TIME_WINDOW = datetime.timedelta(days=365)

value_plots = {}
for language in LANGUAGES:
    print(f"Processing {language}...")
    path_to_lib = f"data/results/{language}"
    
    with open(f"{path_to_lib}/{language}_{language}_post_stats.json") as handle:
        post_stats = json.load(handle)
    sorted_post_stats = sorted(post_stats, key=lambda item:  datetime.datetime.strptime(item['date'], "%Y-%m-%dT%H:%M:%S.%f"))
    
    with open(f"{path_to_lib}/{language}_pairs_first_dates.json") as handle:
        pairs_first_dates = json.load(handle)
    
    with open(f"{path_to_lib}/{language}_all_pairs_dates.json") as handle:
        pairs_dates = json.load(handle)

    with open(f"{path_to_lib}/{language}_{language}_daily_post_stats.json") as handle:
        daily_post_stats = json.load(handle)
    
    annual_post_count = collections.defaultdict(int)
    annual_value_count = {str(year): 0 for year in range(2008, 2024)}


    # get cumulative_daily post_stats
    cumulative_daily_post_stats = {}
    runner = datetime.datetime(2008,1,1).date()
    while runner < datetime.datetime(2024,1,1).date():
        cumulative_daily_post_stats[runner] = 0
        runner += datetime.timedelta(days=1)
    summer = 0
    for k in cumulative_daily_post_stats:
        record = daily_post_stats.get(k.strftime("%Y-%m-%d"), 0)
        summer += record
        cumulative_daily_post_stats[k] = summer

    # Count valuable posts
    for post_stat in tqdm.tqdm(sorted_post_stats):
        year = post_stat["date"][:4]
        annual_post_count[year] += 1
        for imp1, imp2 in itertools.combinations(post_stat["imports"], 2):
            canonical_pair_name = "|".join(sorted([imp1, imp2]))
            if pairs_first_dates[canonical_pair_name]["id"] == post_stat["id"]:
                # novel post, we can assess if it is valuable
                relevant_dates = [
                    dt for dt in pairs_dates[canonical_pair_name]if (
                        datetime.datetime.strptime(dt, "%Y-%m-%d") <
                        datetime.datetime.strptime(pairs_first_dates[canonical_pair_name]["date"], "%Y-%m-%d") + TIME_WINDOW
                        )
                ]
                posts_in_interval = (
                    cumulative_daily_post_stats[
                        min(
                            datetime.datetime(2023,12,31),
                            datetime.datetime.strptime(post_stat["date"], "%Y-%m-%dT%H:%M:%S.%f") + datetime.timedelta(days=365),
                        ).date()
                    ] - 
                    cumulative_daily_post_stats[
                        (datetime.datetime.strptime(post_stat["date"], "%Y-%m-%dT%H:%M:%S.%f") - datetime.timedelta(days=1)).date()
                    ]
                )

            if len(relevant_dates) / max(1, posts_in_interval) > VALUE_THRESHOLD_RATIO:
                annual_value_count[year] += 1
                break

    xs = list(annual_post_count.keys())
    ys = [n/max(1, t) for n, t in zip(annual_value_count.values(), annual_post_count.values())]
    value_plots[language] = (xs, ys)

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(10)
f.set_figwidth(20)
for lang in value_plots:
    ax.plot(*value_plots[lang], "-*", label=lang)
ax.legend()
ax.set_ylabel(f"Probability of a Post being a Valuable Novelty (value ratio: {100 * VALUE_THRESHOLD_RATIO}%)")
ax.grid(alpha = 0.5)
plt.xticks(rotation=70)
ax.set_ylim(ymin=0)
ax.set_xlim(["2010", "2022"])
plt.xticks(size=15)
plt.yticks(size=15)
